In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

In [4]:
urls = ['https://www.rev.com/blog/transcript-tag/andrew-cuomo-transcripts']
for page in range(2, 13):
    url = f'https://www.rev.com/blog/transcript-tag/andrew-cuomo-transcripts/page/{page}'
    urls.append(url)

In [5]:
url

'https://www.rev.com/blog/transcript-tag/andrew-cuomo-transcripts/page/12'

In [6]:
response = requests.get(url)
page = response.text

In [7]:
soup = BeautifulSoup(page, 'html5lib')

In [19]:
link_list = []
for url in urls: 
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, 'html5lib')
    head = soup.find('div', class_="fl-post-column")
    link = head.find('a')['href']
    link_list.append(link)
    while head.findNext('div', class_='fl-post-column') != None:
        head = head.findNext('div', class_='fl-post-column')
        link = head.find('a')['href']
        link_list.append(link)
        

In [56]:
list(enumerate(link_list))

[(0,
  'https://www.rev.com/blog/transcripts/new-york-gov-andrew-cuomo-press-conference-transcript-october-26'),
 (1,
  'https://www.rev.com/blog/transcripts/new-york-gov-andrew-cuomo-press-conference-transcript-october-21'),
 (2,
  'https://www.rev.com/blog/transcripts/ny-gov-andrew-cuomo-teleconference-briefing-transcript-october-20'),
 (3,
  'https://www.rev.com/blog/transcripts/new-york-gov-andrew-cuomo-press-conference-transcript-october-18'),
 (4,
  'https://www.rev.com/blog/transcripts/ny-gov-andrew-cuomo-conference-call-transcript-october-14'),
 (5,
  'https://www.rev.com/blog/transcripts/ny-gov-andrew-cuomo-conference-call-transcript-october-9'),
 (6,
  'https://www.rev.com/blog/transcripts/new-york-gov-andrew-cuomo-press-conference-transcript-october-6'),
 (7,
  'https://www.rev.com/blog/transcripts/new-york-gov-andrew-cuomo-press-conference-transcript-october-5'),
 (8,
  'https://www.rev.com/blog/transcripts/ny-gov-andrew-cuomo-conference-call-transcript-october-1'),
 (9,
  

In [77]:
text_df = []
count=0
for link in link_list:
    response = requests.get(link)
    page = response.text
    soup = BeautifulSoup(page, 'html5lib')
    date = soup.find('div', class_='fl-rich-text').find('p').text
    head = soup.find('div', class_ = 'fl-callout-text') 
    paragraph = head.find('p')
    vid_link = paragraph.find('a')['href']
    paragraph_text = paragraph.text
    par_split = paragraph_text.split(": (")
    par_split_again = par_split[1].split(')\n')
    item = [link, vid_link, date, par_split[0], par_split_again[0], par_split_again[1]]
    text_df.append(item)
    count+=1
    print(count)
    try:
        while (paragraph.findNext('p')!=None) & (paragraph.findNext('p').find('a')['href']!=None):
            paragraph = paragraph.findNext('p')
            vid_link = paragraph.find('a')['href']
            paragraph_text = paragraph.text
            par_split = paragraph_text.split(": (")
            par_split_again = par_split[1].split(')\n')
            try:
                item = [link, vid_link, date, par_split[0], par_split_again[0], par_split_again[1]]
            except IndexError:
                item = [link, vid_link, date, par_split[0], par_split_again[0], ""]
            text_df.append(item)
    except TypeError:
        None

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123


In [139]:
df = pd.DataFrame(text_df, columns = ['url', 'url_time', 'date', 'speaker', 'time', 'text']).copy()

In [140]:
df['datetime']=pd.to_datetime(df['date'])

In [141]:
df_per_comment = df.copy()

In [142]:
df_per_day = df[['url', 'date', 'datetime', 'text', 'speaker']].copy()

In [143]:
df_per_day['speaker_text'] = df_per_day['speaker'].values+' '+df_per_day['text'].values

In [144]:
df_per_day.drop(['text', 'speaker', 'date'], axis=1, inplace=True)

In [145]:
df_day_ag = df_per_day.groupby(['url', 'datetime']).sum()

In [125]:
df_day_ag.to_csv('/Users/gavin/Documents/Metis/coursework/project_4/dataframes/daily_agg.csv')

In [126]:
df_per_comment.to_csv('/Users/gavin/documents/metis/coursework/project_4/dataframes/per_comment.csv')

In [146]:
df_per_month = df.copy()

In [147]:
df_per_month.drop(['url','url_time', 'date', 'speaker', 'time'], axis = 1, inplace = True)

In [148]:
df_per_month['month'] = df_per_month.datetime.dt.month

In [149]:
df_per_month.drop('datetime', axis=1, inplace = True)

In [172]:
df_per_month['text_fix']=df_per_month['text']+' '

In [173]:
df_per_month['text'] = df_per_month['text_fix']
df_per_month.drop('text_fix', axis=1, inplace=True)

In [174]:
df_per_month = df_per_month.groupby('month').sum()

In [182]:
df_per_month.to_csv('/Users/gavin/documents/metis/coursework/project_4/dataframes/text_by_month.csv')

In [152]:
df_per_week = df.copy()

In [154]:
df_per_week.drop(['url', 'url_time', 'date', 'speaker', 'time'], axis=1, inplace = True)

In [170]:
df_per_week['week']=df_per_week.datetime.dt.strftime("%V")

In [177]:
df_per_week['text'] = df_per_week['text']+' '

In [178]:
df_per_week = df_per_week.groupby('week').sum()

In [179]:
df_per_week.head()

,text
week,
11,"Good afternoon. It’s the little things, like m..."
12,Happy Friday. Almost time for the weekend. Que...
13,Happy Sunday. It’s good see all your smiling f...
14,"Good morning, happy Sunday. For those of you w..."
15,"In this beautiful day, let’s start with the go..."


In [181]:
df_per_week.to_csv('/Users/gavin/documents/metis/coursework/project_4/dataframes/text_by_week.csv')